In [89]:
import pandas as pd
from influxdb_client import InfluxDBClient, QueryApi
import os
import statsmodels.api as sm
import numpy as np
from dotenv import load_dotenv
from flask import Flask, jsonify



INFLUXDB_TOKEN = os.getenv("INFLUXDB_TOKEN")
INFLUXDB_ORG = os.getenv("INFLUXDB_ORG")
INFLUXDB_BUCKET = os.getenv("INFLUXDB_BUCKET")
INFLUXDB_URL = os.getenv("INFLUXDB_URL")


client = InfluxDBClient(url="http://localhost:8086", token=INFLUXDB_TOKEN, org=INFLUXDB_ORG, timeout=30_000)

query_api = client.query_api()
query = 'from(bucket: "crypto")\
|> range(start: -1y)\
|> filter(fn: (r) => r["_measurement"] == "BTCUSDT")'
result = query_api.query(org=INFLUXDB_ORG, query=query)



results = []
for table in result:
    for record in table.records:
        results.append((record.get_field(), record.get_value(),record.get_time()))
    
data=pd.DataFrame(results)






In [90]:
data

,0,1,2
0,close,41648.99000,2022-02-06 09:00:00+00:00
1,close,41578.68000,2022-02-06 10:00:00+00:00
2,close,41522.00000,2022-02-06 11:00:00+00:00
3,close,41610.99000,2022-02-06 12:00:00+00:00
4,close,41692.35000,2022-02-06 13:00:00+00:00
...,...,...,...
87275,volume,5484.38416,2023-02-04 20:00:00+00:00
87276,volume,4614.64216,2023-02-04 21:00:00+00:00
87277,volume,4638.30091,2023-02-04 22:00:00+00:00
87278,volume,9471.40173,2023-02-04 23:00:00+00:00


In [91]:
data=data.rename({0:"_field",1:"_value",2:"_time"},axis=1)
data['_time'] = pd.to_datetime(data['_time'],format='%Y-%m-%dT%H:%M:%SZ',errors='coerce')
data['_value']=pd.to_numeric(data['_value'],errors='coerce',downcast="float")
data.dropna(inplace=True)
data=pd.pivot(data,index='_time',columns='_field',values='_value')
data=data.reset_index()

In [92]:
data=data.iloc[:-100,:]

In [93]:



exog=data.copy()
exog.drop('high',axis=1,inplace=True)

exog['_time']=pd.DatetimeIndex(exog.reset_index()['_time']) + pd.DateOffset(hours=1)
data=data[1:]
last_exog=exog.iloc[-1]
exog=exog.iloc[:-1]

data.set_index('_time',inplace=True)
exog.set_index('_time',inplace=True)


In [94]:
endog=data['high'].values.reshape(-1,1)
model = sm.tsa.arima.ARIMA(endog=endog[:-1],exog=np.array(exog[:-1],dtype='float'), order=(5,3,3))
results = model.fit(method='statespace')
result=results.forecast(steps=1,exog=exog.iloc[-1])
result=result[0]

/Users/dariodm/opt/anaconda3/envs/proj_nuovo/lib/python3.10/site-packages/statsmodels/tsa/statespace/sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'
/Users/dariodm/opt/anaconda3/envs/proj_nuovo/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


In [95]:
result

23127.2008936255